# Middleware
---

在`2. Modules Need to be Installed`中安装了一些modules, mongoose都知道是MongoDB的ODM. 那剩下两个呢? 这就要介绍一下express中middleware的概念(middleware就是函数, 准确来说是在你的程序route不同的http请求之前执行的函数, 比如http请求刚来到, 你需要验证一下cookie, 解码一下http请求的编码等):

Middleware functions are functions that have access to the request object (**req**), the response object (**res**), and the **next** middleware function in the application’s request-response cycle. The next middleware function is commonly denoted by a variable named **next**. (对于next会有疑惑, 就是说你可以事先设置多个middleware function, 然后这些函数是不是需要一个一个的按顺序执行? 对, 如果你有三个middleware functions, 那么你必须在前两个functions体里调用函数`next()`, 目的是为了执行下一个middleware function, 否则这个http请求就一直被hanging, 不会有接下来的事发生了, 一会在下面看个例子, 你就知道next的作用了)

If the current middleware function **does not** end the request-response cycle, it must call `next()` to pass control to the next middleware function. Otherwise, the request will be left hanging.

## Example
首先, 添加middleware function很简单, 使用`app.use()`函数进行添加即可. 

Express apps have a `use()` function. This function adds a new `middleware` to the app.

```js
const app = require('express')();

const port = 3000;
app.use((req, res, next) => {
    console.log(`${req.method} ${req.url}`);
    
    // 一定别忘记在middleware function中调用next()函数, 以便执行function stack中的下一个函数
    next();
});

app.get('/', (req, res) => {
    res.send('Hello World, from express');
    console.log('Received a get http request');
});

app.listen(port, () => console.log('start listening...'));

// 发送http请求, terminal打印的内容如下, 可以看出, middleware function先于routes的那些函数调用
start listening...
GET /
Received a get http request
```

下面这句话真是直接道明了别人几段话都解释不清的意思:

> Each app.use(middleware) is called every time a request is sent to the server.

# The Middleware Stack
---

之所以把这段单独分为一节, 是因为它很重要, 可以增加对express applacation的理解. 

In Express, everything is middleware. Internally, **an Express app has a middleware stack**, and **calling use() adds a new layer to the stack**. Functions that define route handlers, like get() and post() **also** add layers to the stack. Express executes the middleware stack **in order**, so the order in which you call use() matters. 

所以这句话说的没毛病:
> An Express application is essentially a series of middleware function calls.

For example, one of the most common middleware functions is the [cors middleware](https://masteringjs.io/tutorials/express/cors), which attaches CORS headers to your Express HTTP responses. Make sure you call `app.use(cors())` before defining any route handlers or anything else that sends an HTTP response, otherwise you won't get CORS headers!

```js
const app = require('express')();

// This response will **NOT** have CORS headers, because order matters.
// Express will run the CORS middleware _after_ this route handler.
// 这句也就是所谓的define route handlers
app.get('/nocors', (req, res) => {
  res.send('ok');
});

app.use(require('cors')());

// This response will have CORS headers, because this route handler
// is after the CORS middleware in the middleware list.
app.get('/cors', (req, res) => {
  res.send('ok');
});
```

Another common middleware function is Express `body-parser`. This middleware is responsible for parsing the request body and **setting** the req.body property. Make sure you call app.use(express.json()) before using req.body, otherwise it will be undefined!

```js
const app = require('express')();

// `body` will always be `undefined` in the HTTP response, because
// Express will run the JSON body parser _after_ this route handler.
app.post('/nobody', (req, res) => {
  res.json({ body: req.body });
});

app.use(express.json());

// `body` will contain the inbound request body.
app.post('/body', (req, res) => {
  res.json({ body: req.body });
});
```

https://masteringjs.io/tutorials/express/app-use

# The `path` Parameter
---

Although the `use()` function is typically called with only 1 parameter, you can also pass it a `path` that tells Express to only execute the given middleware when it receives a request for a URL that starts with the given path.

```js
const app = require('express')();

app.use('/cors', require('cors')());

// This response will **NOT** have CORS headers, because the path '/nocors'
// doesn't start with '/cors'
app.get('/nocors', (req, res) => {
  res.send('ok');
});

// This response will have CORS headers
app.get('/cors', (req, res) => {
  res.send('ok');
});

// This response will also have CORS headers, because '/cors/test' starts
// with '/cors'
app.get('/cors/test', (req, res) => {
  res.send('ok');
});
```

好好想想什么是rest api里面的path, 什么又是query parameters, 不要弄混. 